In [2]:
import xarray as xr
import numpy as np
from geocube.vector import vectorize
da = xr.open_dataset(r"D:\Datasets\Köppen-Geiger Climate Classification\KG_1986-2010.grd", engine="rasterio").band_data.sel(band=1)

# To geopandas
gdf = vectorize(da)

# Drop ocean
gdf = gdf[gdf.band_data != 32]

# Assign categories
categories_32 = ['Af', 'Am', 'As', 'Aw', 'BSh', 'BSk', 'BWh', 'BWk', 'Cfa', 'Cfb','Cfc', 'Csa', 'Csb', 'Csc', 'Cwa','Cwb', 'Cwc', 'Dfa', 'Dfb', 'Dfc','Dfd', 'Dsa', 'Dsb', 'Dsc', 'Dsd','Dwa', 'Dwb', 'Dwc', 'Dwd', 'EF','ET', 'Ocean']
labels_32 = dict(zip(range(1, 32+1), categories_32))

labels_5 = {"A":"Equatorial", "B":"Arid", "C":"Warm Temperate", "D":"Boreal", "E":"Polar"}

gdf['climate_band_32'] = gdf['band_data'].map(labels_32)
gdf['climate_band_5_data'] = gdf['climate_band_32'].str[0]
gdf['climate_band_5'] = gdf['climate_band_5_data'].map(labels_5)

# Show
# gdf.plot(column="climate_band_5")

In [3]:
import pandas as pd
import geopandas as gpd

df = pd.read_stata(r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_in\DHS\DHSBirthsGlobalAnalysis_05142024.dta")
gdf_dhs = df[["ID_HH","LATNUM","LONGNUM"]].drop_duplicates(subset="ID_HH")
gdf_dhs = gpd.GeoDataFrame(gdf_dhs, geometry=gpd.points_from_xy(gdf_dhs["LONGNUM"], gdf_dhs["LATNUM"]))
# gdf_dhs.plot()

In [8]:
gdf_dhs = gdf_dhs.set_crs("epsg:4326", allow_override=True)
gdf = gdf.set_crs("epsg:4326", allow_override=True)
gdf_dhs_climatebands = gdf_dhs.sjoin(gdf)

In [12]:
gdf_dhs_climatebands.to_csv(r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_proc\DHS_climate_bands_assigned.csv")

# Assign to final dataset

In [15]:
# FIXME: Esto no deberia estar, deberia estar en el 03 de stata

In [14]:
out_dhs = pd.read_csv(r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_out\DHSBirthsGlobal&ClimateShocks_v4.csv")


,ID_R,ID_CB,chb_year,chb_month,child_agedeath_30d,child_agedeath_30d3m,child_agedeath_3m6m,child_agedeath_6m12m,child_agedeath_12m,child_fem,...,stdm_t_30d_avg_neg,stdm_t_30d_avg_sq_pos,stdm_t_30d_avg_sq_neg,stdm_t_2m12m_avg_sq,stdm_t_2m12m_avg_dpos,stdm_t_2m12m_avg_dneg,stdm_t_2m12m_avg_pos,stdm_t_2m12m_avg_neg,stdm_t_2m12m_avg_sq_pos,stdm_t_2m12m_avg_sq_neg
0,AL5-1-1-3,AL5-1-1-3-1-1,2000,10,0,0,0,0,0,0,...,0.000000,0.996914,0.000000,0.237739,1,0,0.487585,0.000000,0.237739,0.000000
1,AL5-1-10-4,AL5-1-10-4-1-2,2002,6,0,0,0,0,0,0,...,0.000000,0.337351,0.000000,0.010397,0,1,0.000000,-0.101968,0.000000,0.010397
2,AL5-1-10-4,AL5-1-10-4-1-1,2006,6,0,0,0,0,0,1,...,-0.205585,0.000000,0.042265,0.021400,1,0,0.146287,0.000000,0.021400,0.000000
3,AL5-1-12-3,AL5-1-12-3-1-1,2000,10,0,0,0,0,0,0,...,0.000000,0.996914,0.000000,0.237739,1,0,0.487585,0.000000,0.237739,0.000000
4,AL5-1-14-3,AL5-1-14-3-1-2,1999,7,0,0,0,0,0,0,...,0.000000,0.210259,0.000000,0.051590,1,0,0.227135,0.000000,0.051590,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054974,ZW7-99-8-3,ZW7-99-8-3-99-1,2011,3,0,0,0,0,0,0,...,0.000000,0.602647,0.000000,0.090985,1,0,0.301636,0.000000,0.090985,0.000000
3054975,ZW7-99-9-1,ZW7-99-9-1-99-2,2008,4,0,0,0,0,0,1,...,-0.228849,0.000000,0.052372,0.001061,1,0,0.032573,0.000000,0.001061,0.000000
3054976,ZW7-99-9-1,ZW7-99-9-1-99-2,2008,4,0,0,0,0,0,1,...,-0.228849,0.000000,0.052372,0.001061,1,0,0.032573,0.000000,0.001061,0.000000
3054977,ZW7-99-9-1,ZW7-99-9-1-99-1,2011,11,0,0,0,0,0,0,...,0.000000,0.026934,0.000000,0.588900,1,0,0.767398,0.000000,0.588900,0.000000


In [17]:
gdf_dhs_climatebands[["ID_HH", "climate_band_32", "climate_band_5"]]

,ID_HH,climate_band_32,climate_band_5
0,AL5-227-3,Csa,Warm Temperate
1,AL5-47-15,Csb,Warm Temperate
2,AL5-185-10,Cfb,Warm Temperate
3,AL5-51-6,Cfb,Warm Temperate
4,AL5-92-3,Csa,Warm Temperate
...,...,...,...
5259224,ZW7-111-24,Cwa,Warm Temperate
5259253,ZW7-123-27,Cwb,Warm Temperate
5259271,ZW7-131-4,BSh,Arid
5259349,ZW7-131-18,BSh,Arid


In [18]:
merged = out_dhs.merge(gdf_dhs_climatebands[["ID_HH", "climate_band_32", "climate_band_5"]], on="ID_HH", indicator=True, how="outer")

In [21]:
merged._merge.value_counts()

_merge
both          3018281
right_only     457418
left_only       36698
Name: count, dtype: int64

In [23]:
merged[merged._merge=="both"].drop(columns="_merge").to_csv(r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_out\DHSBirthsGlobal&ClimateShocks_v4_clibands.csv")

In [24]:
merged

,ID_R,ID_CB,chb_year,chb_month,child_agedeath_30d,child_agedeath_30d3m,child_agedeath_3m6m,child_agedeath_6m12m,child_agedeath_12m,child_fem,...,stdm_t_2m12m_avg_sq,stdm_t_2m12m_avg_dpos,stdm_t_2m12m_avg_dneg,stdm_t_2m12m_avg_pos,stdm_t_2m12m_avg_neg,stdm_t_2m12m_avg_sq_pos,stdm_t_2m12m_avg_sq_neg,climate_band_32,climate_band_5,_merge
0,AL5-1-1-3,AL5-1-1-3-1-1,2000.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.237739,1.0,0.0,0.487585,0.000000,0.237739,0.000000,Csa,Warm Temperate,both
1,AL5-1-10-4,AL5-1-10-4-1-2,2002.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.010397,0.0,1.0,0.000000,-0.101968,0.000000,0.010397,Csa,Warm Temperate,both
2,AL5-1-10-4,AL5-1-10-4-1-1,2006.0,6.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.021400,1.0,0.0,0.146287,0.000000,0.021400,0.000000,Csa,Warm Temperate,both
3,AL5-1-12-3,AL5-1-12-3-1-1,2000.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.237739,1.0,0.0,0.487585,0.000000,0.237739,0.000000,Csa,Warm Temperate,both
4,AL5-1-14-3,AL5-1-14-3-1-2,1999.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.051590,1.0,0.0,0.227135,0.000000,0.051590,0.000000,Csa,Warm Temperate,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3512392,ZW7-99-8-3,ZW7-99-8-3-99-1,2011.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.090985,1.0,0.0,0.301636,0.000000,0.090985,0.000000,BSh,Arid,both
3512393,ZW7-99-9-1,ZW7-99-9-1-99-2,2008.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.001061,1.0,0.0,0.032573,0.000000,0.001061,0.000000,BSh,Arid,both
3512394,ZW7-99-9-1,ZW7-99-9-1-99-2,2008.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.001061,1.0,0.0,0.032573,0.000000,0.001061,0.000000,BSh,Arid,both
3512395,ZW7-99-9-1,ZW7-99-9-1-99-1,2011.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.588900,1.0,0.0,0.767398,0.000000,0.588900,0.000000,BSh,Arid,both
